In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 세팅
plt.style.use('seaborn')
sns.set(font_scale=2.5)

# 결측값을 보여주는 라이브러리
import missingno as msno

# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%matplotlib inline

ModuleNotFoundError: No module named 'missingno'

In [ ]:
# 데이터 정리 -> NULL 값들 확인 -> EDA: 각각의 특성들 분석 
# -> feature engineering -> 모델 생성 -> 모델 학습 및 생성 -> 모델 평가

In [ ]:
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
df_train.head()

In [ ]:
# 각 column의 간단한 통계적 분석\
# 총 인원 수가 891명인데 Age는 714명으로 count됨.
# -> Age column에 missing value가 있다
df_train.describe()

In [ ]:
df_train.shape

In [ ]:
df_test.describe()

In [ ]:
# 각 column의 NaN 값들 비율 확인
for col in df_train.columns:
    # {:>10} : 10자리 표현. 오른쪽 정렬
    # {:.2f} : 소수점 2자리까지 표현
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

In [ ]:
# 각 column의 NaN 값들 비율 확인
for col in df_test.columns:
    # {:>10} : 10자리 표현. 오른쪽 정렬
    # {:.2f} : 소수점 2자리까지 표현 
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

In [ ]:
# iloc: index location. 내가 원하는 행과 열을 가져올 수 있음
# 빈칸이 NULL값. 어느 위치에 NULL 데이터가 존재하는지
msno.matrix(df=df_train.iloc[:,:], figsize=(8,8), color=(0.8, 0.5, 0.2))

In [ ]:
# target label 확인
# target label이 얼마나 balance 있게 분포되어 있는가 
# target label이 어떤 distribution을 갖는지 반드시 확인을 해야 함.

# 그래프를 그리기 위한 도화지를 준비하는 과정
# subplots()를 통해 도화지에 구획을 나눔. 아래의 예는 1행 2열로 구획을 나눔 + figsize는 도화지 전체의 크기
# f는 전체 도화지를 지칭하며 ax는 각각의 구획을 지칭. 리스트 형태이며 인덱스로 접근 가능 ex) ax[0], ax[1]
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# value_counts(): 각 레이블의 개수를 반환. type은 Series -> plot을 가지고 있음 (plt와의 호환성)
# pie(): plot을 파이 그래프로 만듦
# explode: 각 부채꼴이 중심에서 벗어나는 정도. 아래는 부채꼴이 2개니까 2개의 요소를 가지고 있는 리스트
# autopct: 부채꼴 안에 표시될 숫자의 형식
# ax: 파이 그래프를 그릴 구획 지정
df_train['Survived'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')

# seaborn의 count plot 생성
sns.countplot('Survived', data=df_train, ax=ax[1])
ax[1].set_title('Count plot - Survived')
plt.show()

# target label이 balance하게 분포하여 있다.

# EDA

In [ ]:
# EDA: Exploratory Data Analysis
# feature들 간의 상관관계, 어떤 feature가 중요한지, feature들의 insight를 얻는 중요한 부분
# 그림을 해석하는 능력을 기르자.
# 내가 원하는 주장을 할 수 있는 그림을 그리자. 

## 2.1 Pclass

In [ ]:
df_train['Pclass'].unique()

In [ ]:
# Pclass에 따른 생존율의 변화
# 각 Pclass에 해당되는 사람들: Pclass와 Survived column만 가져와서 Pclass로 묶음 -> count()
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).count()

In [ ]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).mean() # 생존율

In [ ]:
# 각 Pclass에 해당되는 사람들 중 생존한 사람들 -> sum()
df_train[['Pclass', 'Survived']].groupby(['Pclass']).sum()

In [ ]:
# crosstab()을 사용하여 Pclass와 Survived column을 합친 표를 생성. margins=: All의 여부
pd.crosstab(df_train['Pclass'], df_train['Survived'], margins=True).style.background_gradient(cmap='cool')

In [ ]:
# 각 클래스별로 생존율 확인
# Pclass가 높을수록 생존율이 높음
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar()

In [ ]:
y_position = 1.02
f,ax = plt.subplots(1, 2, figsize=(18, 8))
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32', '#FFDF00', '#D3D3D3'], ax=ax[0])
ax[0].set_title('Number of passengers By Pclass', y=y_position)
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y=y_position)
plt.show()

# => Pclass가 높을수록 생존 확률이 높다

## 2.2 Sex

In [ ]:
# 성별에 따른 생존율 변화

f, ax = plt.subplots(1, 2, figsize=(18, 8))

df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=True).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Sex: Survived vs Dead')
plt.show()

# 여성의 생존 확률이 현저히 높음

In [ ]:
# sex와 survived에 대해 크로스 테이블 생성
pd.crosstab(df_train['Sex'], df_train['Survived'], margins=True)

## 2.2 Both Sex and Pclass 

In [ ]:
# sex와 pclass를 모두 고려한 생존율 그래프 생성 -> factorplot 이용
sns.factorplot('Pclass', 'Survived', hue='Sex', data=df_train, size=6, aspect=1.5)

In [ ]:
# 클래스가 높을수록 생존율이 높아짐. 클래스에 상관 없이 여성은 생존율이 높음
# 그래프의 세로 막대: 에러바
sns.factorplot(x='Sex', y='Survived', hue='Pclass', data=df_train, saturation=.5, size=9, aspect=1)

## 2.3 Age

In [ ]:
print("제일 나이 많은 탑승객: {:.1f} years".format(df_train['Age'].max()))
print("제일 어린 탑승객: {:.1f} years".format(df_train['Age'].min()))
print("탑승객 평균 나이: {:.1f} years".format(df_train['Age'].mean()))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5))

# kdeplot(kernel density estimation): 히스토그램을 밀도 함수로 바꿔서 표현해주는 그래프. 
# 히스토그램으로는 알기 어려운 데이터의 분포를 커널 함수를 통해 smooth하게 표현해줌 -> practical하게 표현해줌.
# 생존한 사람들의 Age column만을 뽑아 kdeplot으로 표현
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax=ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax=ax)
plt.legend(['Survived == 1', 'Survived == 0'])
plt.show()

# => 나이가 어릴수록 생존할 확률이 높아진다

In [ ]:
# 생존한 사람들의 Age column만을 뽑아 히스토그램으로 표현
df_train[df_train['Survived'] == 1]['Age'].hist()

In [ ]:
# Age와 Pclass 간의 상관관계
plt.figure(figsize=(8, 6))
df_train['Age'][df_train['Pclass'] == 1].plot(kind='kde')
df_train['Age'][df_train['Pclass'] == 2].plot(kind='kde')
df_train['Age'][df_train['Pclass'] == 3].plot(kind='kde')

plt.legend(['Pclass == 1', 'Pclass == 2', 'Pclass == 3'])
plt.xlabel('Age')
plt.title('Age Distribution within classes')

# => 나이가 많을 수록 좋은 좌석에 앉은 경향

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5))

# Pclass가 1인 사람들 중에서 생존한 사람들과 그렇지 않은 사람들의 나이 분포
sns.kdeplot(df_train[(df_train['Survived'] == 0) & (df_train['Pclass'] == 1)]['Age'], ax=ax)
sns.kdeplot(df_train[(df_train['Survived'] == 1) & (df_train['Pclass'] == 1)]['Age'], ax=ax)
plt.legend(['Survived == 0', 'Survived == 1'])
plt.title('1st class')
plt.show()

# => 1st class에서는 젊은 사람들이 더 많이 생존

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5))

# Pclass가 2인 사람들 중에서 생존한 사람들과 그렇지 않은 사람들의 나이 분포
sns.kdeplot(df_train[(df_train['Survived'] == 0) & (df_train['Pclass'] == 2)]['Age'], ax=ax)
sns.kdeplot(df_train[(df_train['Survived'] == 1) & (df_train['Pclass'] == 2)]['Age'], ax=ax)
plt.legend(['Survived == 0', 'Survived == 1'])
plt.title('2nd class')
plt.show()

# => 2nd class도 나이가 어릴수록 생존자가 많음

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5))

# Pclass가 3인 사람들 중에서 생존한 사람들과 그렇지 않은 사람들의 나이 분포
sns.kdeplot(df_train[(df_train['Survived'] == 0) & (df_train['Pclass'] == 3)]['Age'], ax=ax)
sns.kdeplot(df_train[(df_train['Survived'] == 1) & (df_train['Pclass'] == 3)]['Age'], ax=ax)
plt.legend(['Survived == 0', 'Survived == 1'])
plt.show()

# => 3rd class도 나이가 어릴수록 생존자가 많음

In [ ]:
# 그러면 정말 어린 사람들의 생존율이 더 높은가?
cumulate_survival_ratio = []

for i in range(1, 80):
    # 나이가 i보다 적은 사람들 전부 중에서 생존한 사람들의 비율을 리스트에 append
    num_person_lower_i = df_train[df_train['Age'] < i]['Survived']
    cumulate_survival_ratio.append(num_person_lower_i.sum() / len(num_person_lower_i))
    

plt.figure(figsize=(7, 7))
plt.plot(cumulate_survival_ratio)
plt.title('Survival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age(0-x)')
plt.show()

# => 나이가 어릴수록 생존 확률이 높다!

## Pclass, Sex, Age

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# Pclass와 Age를 기준으로 Survived에 대한 violinplot 생성
sns.violinplot('Pclass', 'Age', hue='Survived', data=df_train, scale='count', split=True, ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0, 110, 10))

# Sex와 Age를 기준으로 Survived에 대한 violineplot 생성
sns.violinplot('Sex', 'Age', hue='Survived', data=df_train, scale='count', split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0, 110, 10))
plt.show()